# BOT using the Envelopes Strategy

In [36]:
class userVals():
    dev = 0.002
    length = 45
    count = 40
    key = '4900a27fe17ab573d490ce234465329f-6633f44d08bf25b48ceaacd2c7917674'
    units = 2000
    accountID = '101-004-9949998-001'
    pair="EUR_USD"
    params = {
        "count": length,
        "granularity": "M15"
    }
    
    waiting_time = 900 #seconds in the granularity
    pip_ladder = 10

In [37]:
import requests as r
import numpy as np
#from __init__ import userVals

from oandapyV20 import API
import oandapyV20.endpoints.instruments as instruments

class user1():
    client = API(access_token=userVals.key)
    o = instruments.InstrumentsCandles(instrument=userVals.pair,
        params=userVals.params)

class candleLogic():
    def OHLC(self, data):
        # Call imported user1 class
        user1.client.request(user1.o)
        candles = user1.o.response.get("candles")
        candleData = candles[data].get("mid")

        # OHLC variables to return in array
        o = candleData.get("o")
        h = candleData.get("h")
        l = candleData.get("l")
        c = candleData.get("c")
        return float(o), float(h), float(l), float(c)

    # Define clean function routes for returning proper data
    def Open(self, data):
        return self.OHLC(data)[0]

    def High(self, data):
        return self.OHLC(data)[1]

    def Low(self, data):
        return self.OHLC(data)[2]

    def Close(self, data):
        return self.OHLC(data)[3]

    def getData(self):
        self.numList = []
        for x in range(0, userVals.length):
            self.numList.append(self.Close(x))
        return self.numList

In [38]:
# Strategy logic & indicators go here
class strategyLogic():
    def ENV(self, rates, period, deviation, length):
        self.UL = []
        self.LL = []
        self.ML = []

        self.SMA = sum(rates[(length-period):length]) / period
    
        self.uls = self.SMA + (self.SMA*deviation)
        self.lls = self.SMA - (self.SMA*deviation)
        self.mls = self.SMA
        
        self.UL.append(self.uls)
        self.ML.append(self.mls)
        self.LL.append(self.lls)
        
        return self.UL, self.ML, self.LL 

In [39]:
# Oanda Packages
from oandapyV20 import API
import oandapyV20
from oandapyV20.contrib.requests import MarketOrderRequest
#from oandapyV20.contrib.requests import TakeProfitDetails, StopLossDetails
import oandapyV20.endpoints.orders as orders
import oandapyV20.endpoints.accounts as accounts
import oandapyV20.endpoints.positions as positions

from datetime import datetime, timedelta, timezone
import dateutil.parser

class trading():
    # state management
    def __init__(self):
        self.status = "Not Trading"
        self.currentTrade = ""
        self.kill = False # <----- This is a kill switch. If it is true, the bot will shut down.

        #initialize data channel
        s = strategyLogic()  
        c = candleLogic()
        self.data = c.getData()         

        #Initialize Indicators
        self.currentClose = self.data[-1]
        self.UL, self.ML, self.LL = s.ENV(rates=self.data, period=userVals.count, deviation=userVals.dev, length=userVals.length)
        self.currentUL = self.UL[-1]
        self.currentLL = self.LL[-1]
        self.currentML = self.ML[-1]

    # Entry/Exit confirmations
    def enterLong(self):
        if (self.currentClose < self.currentLL): return True
        return False

    def enterShort(self): 
        if (self.currentClose > self.currentUL): return True 
        return False

    # Check account for how many open trades
    def getTrades(self):
        r = accounts.AccountDetails(userVals.accountID)
        client = API(access_token=userVals.key)
        rv = client.request(r)
        self.details = rv.get('account')
        return self.details.get('openTradeCount')
    
    # Check size of LONG open trades
    def getSizeTradesLong(self):
        r = accounts.AccountDetails(userVals.accountID)
        client = API(access_token=userVals.key)
        rv = client.request(r)
        self.details = rv.get('account')
        self.details = self.details.get('positions')[0]['long']['units']
        return float(self.details)
    
    # Check size of SHORT open trades
    def getSizeTradesShort(self):
        r = accounts.AccountDetails(userVals.accountID)
        client = API(access_token=userVals.key)
        rv = client.request(r)
        self.details = rv.get('account')
        self.details = self.details.get('positions')[0]['short']['units']
        return float(self.details)
    
    # Check avg price of LONG open trades
    #def getAvgPriceLong(self):
    #    r = accounts.AccountDetails(userVals.accountID)
    #    client = API(access_token=userVals.key)
    #    rv = client.request(r)
    #    self.details = rv.get('account')
    #    self.details = self.details.get('positions')[0]['long']['averagePrice']
    #    return float(self.details)
    #
    ## Check avg price of SHORT open trades
    #def getAvgPriceShort(self):
    #    r = accounts.AccountDetails(userVals.accountID)
    #    client = API(access_token=userVals.key)
    #    rv = client.request(r)
    #    self.details = rv.get('account')
    #    self.details = self.details.get('positions')[0]['short']['averagePrice']
    #    return float(self.details)
    
    # Get entry price of the last trade
    def getPrice(self):
        r = accounts.AccountDetails(userVals.accountID)
        client = API(access_token=userVals.key)
        rv = client.request(r)
        self.dettagliTrade = rv.get('account')
        self.dettagliTrade = self.dettagliTrade.get('trades')
        self.num=int(len(self.dettagliTrade)-1)
        self.entryPrice = self.dettagliTrade[self.num]["price"]
        return float(self.entryPrice)
    
    # Get entry size of the last trade
    def getLastSize(self):
        r = accounts.AccountDetails(userVals.accountID)
        client = API(access_token=userVals.key)
        rv = client.request(r)
        self.dettagliTrade = rv.get('account')
        self.dettagliTrade = self.dettagliTrade.get('trades')
        self.num=int(len(self.dettagliTrade)-1)
        self.lastSize = self.dettagliTrade[self.num]["initialUnits"]
        return float(self.lastSize)
    
    # Check the time of the last entry
    def getDeltaEntryTime(self):
        r = accounts.AccountDetails(userVals.accountID)
        client = API(access_token=userVals.key)
        rv = client.request(r)
        self.openTime = rv.get('account')
        self.openTime = self.openTime.get('trades')
        self.num=int(len(self.openTime)-1)
        self.openTime = self.openTime[self.num]["openTime"]
        self.openTime = dateutil.parser.parse(self.openTime)
        self.adesso = datetime.now(timezone.utc)
        self.diff = (self.adesso - self.openTime).seconds
        return int(self.diff)

    #Define LONG closeout
    def closePosition(self):
        if self.currentTrade == "Long":
            
            data = {"longUnits": "ALL"}
            client = oandapyV20.API(access_token=userVals.key)
            r =positions.PositionClose(accountID=userVals.accountID,instrument=userVals.pair, data=data)
            client.request(r)
            
        elif self.currentTrade == "Short":
            
            data = {"shortUnits": "ALL"}
            client = oandapyV20.API(access_token=userVals.key)
            r =positions.PositionClose(accountID=userVals.accountID,instrument=userVals.pair, data=data)
            client.request(r)
         

    # main trading function
    def main(self):
        
        #initialize data channel
        s = strategyLogic()  
        c = candleLogic()
        self.data = c.getData()         

        #Initialize Indicators
        self.currentClose = self.data[-1]
        self.UL, self.ML, self.LL = s.ENV(rates=self.data, period=userVals.count, deviation=userVals.dev, length=userVals.length)
        self.currentUL = self.UL[-1]
        self.currentLL = self.LL[-1]
        self.currentML = self.ML[-1]

        # Oanda Parameters
        mktOrderLong = MarketOrderRequest(instrument=userVals.pair,
                      units= userVals.units)
        mktOrderShort = MarketOrderRequest(instrument=userVals.pair,
                       units= (userVals.units *-1))

        # Trading Conditions
        #################################################################################
        ###########################      NO OPEN TRADES      ############################
        #################################################################################
        # If there are no open trades and the long condition is True
        if self.getTrades() == 0 and self.enterLong() == True:
            
            api = oandapyV20.API(access_token=userVals.key)
            r = orders.OrderCreate(userVals.accountID, data=mktOrderLong.data)
            api.request(r)
            self.status = "Trading"
            self.currentTrade = "Long"
            print("Trade Executed")
            print(self.getPrice())
            
        # If there are no open trades and the short condition is True
        elif self.getTrades() == 0 and self.enterShort() == True:
            
            api = oandapyV20.API(access_token=userVals.key)
            r = orders.OrderCreate(userVals.accountID, data=mktOrderShort.data)
            api.request(r)
            self.status = "Trading"
            self.currentTrade = "Short"
            print("Trade Executed")
            print(self.getPrice())
                
        # If there are no open trades and the long and short condition is False
        elif self.getTrades() == 0 and self.enterLong() == False and self.enterShort() == False:
            
            print("No Trades Open, Looking for Entry...")

        #################################################################################
        #########################      OPEN TRADES - Short      #########################
        #################################################################################
        
        # If there is at least a short opened and the close condition is True
        elif self.getTrades() > 0 and self.getSizeTradesShort() < 0 and self.currentClose < self.currentLL:

            self.closePosition()
            self.status = "Not Trading"
            print("Trade Exited")
            print(self.getPrice())
            
        # If there is at least a short opened and after 15 minutes the price is higher of 10 pips from the last entry
        elif self.getTrades() > 0 and self.getSizeTradesShort() < 0 and self.enterShort() == True \
            and self.getDeltaEntryTime() > userVals.waiting_time\
            and self.getPrice() + userVals.pip_ladder/10000 < self.currentClose:
            
            api = oandapyV20.API(access_token=userVals.key)
            self.units = getLastSize() * 1.25
            mktOrderShort = MarketOrderRequest(instrument=userVals.pair,
                       units=self.units)
            r = orders.OrderCreate(userVals.accountID, data=mktOrderShort.data)
            api.request(r)
            print("Trade Executed")
            print(self.getPrice())
            
        # If there is at least a short opened and the close condition is False
        elif self.getTrades() > 0 and self.getSizeTradesShort() < 0 and self.currentClose > self.currentLL:
            
            self.currentTrade = "Short"
            self.status = "Trading"
            print("No exits.. Looking")
            print(self.getPrice())
            
        #################################################################################
        #########################      OPEN TRADES - Long      #########################
        #################################################################################
            
         # If there is at least a Long opened and the close condition is True
        elif self.getTrades() > 0 and self.getSizeTradesLong() > 0 and self.currentClose > self.currentUL:

            self.closePosition()
            self.status = "Not Trading"
            print("Trade Exited")
            print(self.getPrice())
            
       # If there is at least a long opened and after 15 minutes the price is lower of 10 pips from the last entry
        elif self.getTrades() > 0 and self.getSizeTradesLong() > 0 and self.enterLong() == True \
            and self.getDeltaEntryTime() > userVals.waiting_time \
            and self.getPrice() - userVals.pip_ladder/10000 > self.currentClose:
            
            api = oandapyV20.API(access_token=userVals.key)
            self.units = getLastSize() * 1.25
            mktOrderLong = MarketOrderRequest(instrument=userVals.pair,
                      units=self.units)
            r = orders.OrderCreate(userVals.accountID, data=mktOrderLong.data)
            api.request(r)
            print("Trade Executed")
            print(self.getPrice())
            
        # If there is at least a long opened and the close condition is False
        elif self.getTrades() > 0 and self.getSizeTradesLong() > 0 and self.currentClose < self.currentUL:
            
            self.currentTrade = "Long"
            self.status = "Trading"
            print("No exits.. Looking")
            print(self.getPrice())
                    
        else:
            self.kill = True
            print("Error, Closing down.")


In [40]:
# Run the bot and kill it if kill switch is engaged
if __name__ == "__main__":
    t = trading()
    while(t.kill == False):
        t.main()

Trade Executed
1.08657
No exits.. Looking
1.08657
No exits.. Looking
1.08657
No exits.. Looking
1.08657
No exits.. Looking
1.08657
No exits.. Looking
1.08657
No exits.. Looking
1.08657
No exits.. Looking
1.08657
No exits.. Looking
1.08657
No exits.. Looking
1.08657
No exits.. Looking
1.08657
No exits.. Looking
1.08657
No exits.. Looking
1.08657
Trade Executed
1.08641
No exits.. Looking
1.08641
No exits.. Looking
1.08641
No exits.. Looking
1.08641
No exits.. Looking
1.08641
No exits.. Looking
1.08641
No exits.. Looking
1.08641
No exits.. Looking
1.08641
No exits.. Looking
1.08641
No exits.. Looking
1.08641
No exits.. Looking
1.08641
Trade Executed
1.08577
No exits.. Looking
1.08577
No exits.. Looking
1.08577
No exits.. Looking
1.08577
No exits.. Looking
1.08577
No exits.. Looking
1.08577
No exits.. Looking
1.08577
No exits.. Looking
1.08577
No exits.. Looking
1.08577
Trade Executed
1.08541
No exits.. Looking
1.08541
No exits.. Looking
1.08541
No exits.. Looking
1.08541
No exits.. Lookin

SSLError: HTTPSConnectionPool(host='api-fxpractice.oanda.com', port=443): Max retries exceeded with url: /v3/accounts/101-004-9949998-001 (Caused by SSLError(SSLError("bad handshake: SysCallError(10060, 'WSAETIMEDOUT')")))